# Valoración Cuantitativa de Project Finance Tokenizado
## Aplicación a Constelación Satelital LEO IoT

**Trabajo de Tesis - Maestría en Finanzas Cuantitativas**

---

### Tabla de Contenidos

1. [Resumen Ejecutivo](#1-resumen-ejecutivo)
2. [Marco Teórico y Metodología](#2-marco-teórico-y-metodología)
3. [Caso Base: Estructura y Flujos](#3-caso-base-estructura-y-flujos)
4. [Simulación Monte Carlo y Análisis de Riesgo](#4-simulación-monte-carlo-y-análisis-de-riesgo)
5. [Pruebas de Estrés](#5-pruebas-de-estrés)
6. [Análisis AMM: Comparación V2 vs V3](#6-análisis-amm-comparación-v2-vs-v3)
7. [Análisis de Cobertura (Hedging)](#7-análisis-de-cobertura-hedging)
8. [Estructura Tradicional vs Tokenizada](#8-estructura-tradicional-vs-tokenizada)
9. [Síntesis WACD](#9-síntesis-wacd)
10. [Análisis de Equity](#10-análisis-de-equity)
11. [Conclusiones y Recomendaciones](#11-conclusiones-y-recomendaciones)
12. [Referencias Bibliográficas](#12-referencias-bibliográficas)
13. [Reproducibilidad](#13-reproducibilidad)

---

## 1. Resumen Ejecutivo

### Pregunta de Investigación

¿Puede la tokenización de instrumentos de deuda en project finance reducir significativamente el costo de capital y mejorar la bancabilidad de proyectos de infraestructura?

### Hallazgos Principales

Este trabajo demuestra cuantitativamente que la tokenización de deuda estructurada en project finance genera beneficios significativos:

| Métrica | Tradicional | Tokenizado + Hedge | Δ |
|---------|-------------|-------------------|---|
| **Probabilidad de Breach** | 24.6% | 2.1% | **-91%** |
| **WACD (all-in)** | 450 bps | 388 bps | **-62 bps** |
| **Prima de Liquidez** | 75 bps | 5 bps | **-70 bps** |
| **Eficiencia de Capital (AMM)** | 1.0x | 5.0x | **+400%** |
| **Bancabilidad** | No | Sí | ✓ |

*Nota: "Tokenizado + Hedge" incluye amortización contingente + AMM V3 + interest rate cap*

### Contribución Principal

El mecanismo de **amortización contingente basada en DSCR** combinado con **liquidez concentrada V3** y **cobertura de tasa (cap)** transforma un proyecto no-bancable (probabilidad de breach 24.6%) en uno bancable (2.1%), reduciendo simultáneamente el costo de deuda en 62 puntos base netos.

### Caso de Estudio

Constelación satelital LEO (Low Earth Orbit) para IoT:
- **Costo total del proyecto:** USD 100M
- **Estructura de deuda:** USD 50M (50% LTV deleveraged)
- **Tenor:** 15 años (4 años de gracia)
- **Tranches:** Senior (60%) / Mezzanine (25%) / Subordinado (15%)

In [ ]:
# Configuración e importaciones
import sys
import importlib
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# Detectar raíz del proyecto (funciona desde cualquier directorio)
def find_project_root():
    """Busca la raíz del proyecto subiendo directorios hasta encontrar pyproject.toml o .git"""
    current = Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / "pyproject.toml").exists() or (parent / ".git").exists():
            return parent
    # Fallback: asumir que estamos en notebooks/
    return current.parent if current.name == "notebooks" else current

PROJECT_ROOT = find_project_root()
sys.path.insert(0, str(PROJECT_ROOT))

# Carga de resultados consolidados
RESULTS_PATH = PROJECT_ROOT / "outputs" / "leo_iot_results.json"
with RESULTS_PATH.open() as f:
    RESULTS = json.load(f)

# Importar paneles interactivos (forzar recarga para desarrollo)
from pftoken.viz import plotly_panels
importlib.reload(plotly_panels)
PANELS = plotly_panels.build_interactive_dashboard(RESULTS)

print(f"✓ Raíz del proyecto: {PROJECT_ROOT}")
print(f"✓ Datos cargados: {RESULTS_PATH}")
print(f"✓ Timestamp: {RESULTS.get('timestamp_utc', 'N/A')}")
print(f"✓ Secciones disponibles: {len(RESULTS.keys())}")
print(f"✓ Paneles recargados: {len(PANELS)}")

## 2. Marco Teórico y Metodología

### 2.1 Fundamentos de Project Finance

El project finance es una estructura de financiamiento donde el repago de la deuda depende exclusivamente de los flujos de caja generados por el proyecto, sin recurso al balance del sponsor (Gatti, 2018). Los elementos fundamentales incluyen:

1. **SPV (Special Purpose Vehicle):** Entidad legal aislada que contiene los activos del proyecto
2. **CFADS (Cash Flow Available for Debt Service):** Flujo de caja después de opex, capex y capital de trabajo
3. **Waterfall de pagos:** Cascada de prioridades para distribución de efectivo
4. **Covenants:** Restricciones contractuales (DSCR, LLCR, LTV)

#### Cálculo de CFADS

Siguiendo la metodología de Yescombe (2013), el CFADS se calcula como:

$$\text{CFADS}_t = \text{Revenue}_t - \text{OPEX}_t - \text{Tax}_t - \Delta\text{WC}_t - \text{RCAPEX}_t$$

Donde:
- $\text{Revenue}_t$: Ingresos operativos (curva S de adopción para IoT satelital)
- $\text{OPEX}_t$: Gastos operativos fijos y variables
- $\text{Tax}_t$: Impuestos con escudo fiscal por pérdidas acumuladas
- $\Delta\text{WC}_t$: Cambio en capital de trabajo
- $\text{RCAPEX}_t$: Capex de mantenimiento/reposición

### 2.2 Modelo de Riesgo Crediticio (Merton)

Utilizamos el modelo estructural de Merton (1974) para estimar la probabilidad de default:

$$DD = \frac{\ln(V_A/D) + (r - 0.5\sigma_A^2)T}{\sigma_A\sqrt{T}}$$

$$PD = \Phi(-DD)$$

Donde:
- $V_A$: Valor de activos = VPN(CFADS)
- $D$: Deuda pendiente acumulada por tranche
- $\sigma_A$: Volatilidad de activos (calibrada por tranche)
- $T$: Horizonte (15 años)

| Tranche | $\sigma_A$ | PD Floor | Recovery | Rating Implícito |
|---------|------------|----------|----------|------------------|
| Senior | 22% | 1.5% | 55% | BBB/A- |
| Mezzanine | 28% | 3.5% | 40% | BB+ |
| Subordinado | 35% | 6.5% | 25% | B |

### 2.3 Simulación Monte Carlo

Se implementó un motor de simulación con 11 variables estocásticas correlacionadas:

| Variable | Distribución | Parámetros | Fuente |
|----------|--------------|------------|--------|
| Revenue Growth | Lognormal | μ=0.065, σ=0.18 | Iridium 10-K (2019-2023) |
| Churn Rate | Beta | α=2.1, β=8.4 | Benchmarks Starlink/OneWeb |
| Rate Shock | Normal | μ=0, σ=0.015 | Fed Funds histórico |
| Launch Failure | Bernoulli | p=0.07 | SpaceX Falcon 9 reliability |
| OPEX Inflation | Lognormal | μ=0.03, σ=0.07 | OECD Telecom (2024) |
| CAPEX Overrun | Lognormal | μ=0, σ=0.20 | Moody's Project Finance (2022) |
| Satellite Degradation | Beta | α=1.5, β=3.5 | IEEE Aerospace (2021) |
| Spectrum Regulatory | Bernoulli | p=0.05 | FCC historical revocations |

**Configuración:**
- Simulaciones: 10,000 paths
- Semilla: 42 (reproducibilidad)
- Variantes antitéticas: Sí (reducción de varianza)
- Correlación: Matriz 11×11 con descomposición de Cholesky

### 2.4 Decomposición de Spreads Tokenizados

Siguiendo el framework de Schär (2021) y Zetzsche et al. (2020), descomponemos el spread en cinco componentes:

$$\text{Spread}_{\text{total}} = \text{Credit} + \text{Liquidity} + \text{Origination} + \text{Servicing} + \text{Infrastructure}$$

La tokenización afecta cada componente:

| Componente | Tradicional | Tokenizado | Δ | Driver |
|------------|-------------|------------|---|--------|
| Liquidez | 75 bps | 5 bps | -70 bps | AMM V3, mercado secundario |
| Transparencia | 25 bps | 5 bps | -20 bps | Reporting on-chain |
| Operacional | 25 bps | 5 bps | -20 bps | Smart contracts |
| Originación | 15 bps | 7.5 bps | -7.5 bps | Automatización |
| Infraestructura | 0 bps | 12 bps | +12 bps | Gas, oráculos, monitoreo |

### 2.5 Modelo AMM (Automated Market Maker)

#### Constant Product (V2)

$$x \cdot y = k$$

El slippage en V2 es proporcional al tamaño de la orden respecto a las reservas.

#### Concentrated Liquidity (V3)

La innovación clave de Uniswap V3 es la liquidez concentrada en rangos de precio:

$$L = \frac{\Delta y}{\sqrt{P_{upper}} - \sqrt{P_{lower}}}$$

Para tokens de deuda que operan cerca de par (±5%), esto permite:
- **5x eficiencia de capital** vs V2
- **83% reducción de slippage** para el mismo capital LP

### 2.6 Cobertura con Derivados de Tasa

Utilizamos el modelo Black-76 para valorar caps de tasa de interés:

$$\text{Caplet} = P(0,T) \cdot [F \cdot N(d_1) - K \cdot N(d_2)]$$

Donde:
- $F$: Tasa forward
- $K$: Strike (4.0%)
- $\sigma$: Volatilidad implícita (20% flat)

---

## 3. Caso Base: Estructura y Flujos

### 3.1 Parámetros del Proyecto

El caso de estudio modela una constelación satelital LEO para servicios IoT:

| Parámetro | Valor | Fuente |
|-----------|-------|--------|
| Costo total | USD 100M | Benchmarks Iridium/OneWeb |
| Deuda | USD 50M (50% LTV) | Deleveraging post WP-08 |
| Equity | USD 50M | Sponsor commitment |
| Tenor | 15 años | Vida útil constelación |
| Gracia | 4 años | Período de construcción/ramp-up |
| DSCR covenant | 1.20x | Standard project finance |

### 3.2 Estructura de Capital

**Estructura Tradicional (60/25/15):**
- Senior: USD 30M @ 6.0% (60%)
- Mezzanine: USD 12.5M @ 8.5% (25%)
- Subordinado: USD 7.5M @ 11.0% (15%)

**Estructura Tokenizada Óptima (55/34/11):**
- Senior: USD 27.5M @ 6.0%
- Mezzanine: USD 17M @ 8.5%
- Subordinado: USD 5.5M @ 11.0%

La optimización hacia más mezzanine se basa en el análisis de frontera eficiente Pareto (WP-10): el tranche mezzanine ofrece mejor retorno ajustado por riesgo (8.5% cupón vs 3.5% PD × 55% LGD = 192 bps EL neto).

### 3.3 Componentes de CFADS

In [ ]:
# Visualización: CFADS vs Servicio de Deuda
PANELS['cfads_vs_debt_service']

**Interpretación:**

- **Años 1-4 (Gracia):** CFADS negativo durante construcción; servicio de deuda cubierto por DSRA inicial (USD 14.3M)
- **Año 5:** Inicio de amortización; CFADS alcanza USD 12.4M vs servicio USD 8.2M
- **Años 6-9:** Pico operacional; CFADS USD 17-24M con amplio colchón sobre servicio
- **Años 10-15:** Declive por degradación satelital; CFADS desciende a USD 12M

### 3.4 Ratios de Cobertura

In [ ]:
# Visualización: LLCR e ICR
PANELS['coverage_ratios']

**Interpretación:**

**LLCR (Loan Life Coverage Ratio) - Cálculo Acumulativo:**

El LLCR se calcula como NPV(CFADS) / Deuda Acumulada, donde la deuda acumulada incluye todos los tranches con igual o mayor senioridad:

- **Senior:** 4.31x vs threshold 1.35x → **220% de margen** (solo deuda Senior en denominador)
- **Mezzanine:** 3.05x vs threshold 1.20x → **154% de margen** (Senior + Mezzanine en denominador)
- **Subordinado:** 2.59x vs threshold 1.05x → **147% de margen** (deuda total en denominador)

**Interpretación correcta:** El tranche Senior tiene el LLCR más alto porque es el más seguro - tiene prioridad de pago y su cobertura solo considera su propia deuda. El Subordinado tiene el LLCR más bajo porque absorbe pérdidas residuales y su cobertura considera toda la deuda del proyecto.

**ICR (Interest Coverage Ratio):**
- Mínimo: -0.16x (año 1, período de gracia con CFADS negativo)
- Mediana: ~8x (años operacionales pico)
- El ICR negativo en años iniciales es esperado y cubierto por DSRA

In [ ]:
# Visualización: Cascada de Pagos
PANELS['waterfall_cascade']

**Mecánica del Waterfall (Gatti, 2018):**

1. **Intereses:** Prioridad absoluta; pagados de CFADS o DSRA
2. **Principal:** Amortización sculpted según cronograma
3. **Reservas (DSRA/MRA):** Fondeo para próximo período
4. **Dividendos:** Solo si todos los covenants satisfechos

**Observaciones clave:**
- Años 1-3: Sin dividendos; CFADS insuficiente
- Año 4: Primeros dividendos (USD 1.3M)
- Años 5-9: Dividendos crecientes hasta USD 17M (año 9)
- Año 15: Liberación de reservas → dividendo final USD 9.9M

---

## 4. Simulación Monte Carlo y Análisis de Riesgo

### 4.1 Configuración de la Simulación

La simulación Monte Carlo genera 10,000 trayectorias de CFADS incorporando las 11 variables estocásticas correlacionadas. El motor utiliza:

- **Vectorización NumPy:** Procesamiento batch para eficiencia (<5 min para 10k sims)
- **Variantes Antitéticas:** Reducción de varianza sin aumentar samples
- **Correlación Cholesky:** Matriz SPD-validada para copula Gaussiana

### 4.2 Fan Chart de DSCR

In [ ]:
# Visualización: Fan Chart DSCR (percentiles Monte Carlo)
PANELS['dscr_fan_chart']

**Interpretación del Fan Chart:**

| Percentil | Año 5 | Año 9 | Año 15 | Interpretación |
|-----------|-------|-------|--------|----------------|
| P95 | 1.97x | 4.48x | 3.15x | Escenario optimista |
| P75 | 1.52x | 3.45x | 2.43x | Cuartil superior |
| **P50** | **1.24x** | **2.81x** | **1.97x** | **Caso base** |
| P25 | 0.99x | 2.26x | 1.59x | Cuartil inferior |
| P05 | 0.68x | 1.55x | 1.09x | Escenario de estrés |

**Hallazgos críticos:**

1. **Año 5 es el cuello de botella:** El P5 (0.68x) está muy por debajo del covenant 1.20x
2. **Recuperación post-gracia:** El DSCR mejora significativamente en años 6-9
3. **Degradación terminal:** Años 14-15 muestran compresión por obsolescencia satelital

### 4.3 Curvas de Breach

La probabilidad de breach acumulada alcanza **46.3% bajo estructura tradicional** concentrada en el año 5 (inicio de amortización).

In [ ]:
# Estadísticas de breach
breach_data = RESULTS['monte_carlo']['breach_probability']
print("=" * 50)
print("ANÁLISIS DE PROBABILIDAD DE BREACH")
print("=" * 50)
print(f"Probabilidad acumulada (año 15): {breach_data['cumulative'][-1]*100:.1f}%")
print(f"Tasa de supervivencia: {breach_data['survival'][-1]*100:.1f}%")
print(f"Año de máximo hazard: Año 5 (tasa hazard {breach_data['hazard'][4]*100:.1f}%)")
print("\nNota: Estructura tradicional sin amortización contingente")

### 4.4 Métricas de Riesgo por Tranche

Aplicando el modelo de Merton con los parámetros calibrados:

In [ ]:
# Tabla de métricas de riesgo
risk = RESULTS['risk_metrics']
print("=" * 60)
print("MÉTRICAS DE RIESGO POR TRANCHE")
print("=" * 60)
print(f"{'Tranche':<15} {'PD Mean':<10} {'LGD':<10} {'EL':<12} {'Capital 99%':<12}")
print("-" * 60)
for tranche, metrics in risk['tranches'].items():
    print(f"{tranche.capitalize():<15} {metrics['pd_mean']*100:.2f}%     {metrics['lgd']*100:.1f}%     {metrics['expected_loss']:.3f}      {metrics['capital_99']:.2f}")

print("\n" + "=" * 60)
print("MÉTRICAS DE PORTAFOLIO")
print("=" * 60)
port = risk['portfolio']
print(f"Expected Loss: {port['expected_loss']:.4f} MUSD")
print(f"VaR 95%: {port['var_95']:.2f} MUSD")
print(f"VaR 99%: {port['var_99']:.2f} MUSD")
print(f"CVaR 95%: {port['cvar_95']:.2f} MUSD")
print(f"CVaR 99%: {port['cvar_99']:.2f} MUSD")

**Interpretación (Artzner et al., 1999 - Medidas Coherentes de Riesgo):**

- **Senior:** PD 1.5%, LGD 35% → EL 0.15 MUSD (bajo riesgo, rating implícito BBB)
- **Mezzanine:** PD 3.5%, LGD 55% → EL 0.24 MUSD (riesgo moderado, rating BB+)
- **Subordinado:** PD 6.5%, LGD 75% → EL 0.37 MUSD (first-loss, rating B)

El CVaR 99% de 11.4 MUSD representa la pérdida esperada en el peor 1% de escenarios, informando los requisitos de capital regulatorio (Basel III/IV).

---

## 5. Pruebas de Estrés

### 5.1 Biblioteca de Escenarios

Se diseñaron 16 escenarios de estrés organizados en categorías:

**Escenarios Base (S1-S6):**
| Código | Nombre | Shocks Principales | Prob. |
|--------|--------|-------------------|-------|
| S1 | Demanda Débil | Revenue -200 bps, Churn +125 bps | 15% |
| S2 | Shock de Tasas | Curva +200 bps paralelo | 10% |
| S3 | Fallo de Lanzamiento | p=100%, revenue delay | 7% |
| S4 | Degradación Operativa | OPEX +100 bps, RCAPEX +150 bps | 12% |
| S5 | Regulatorio Adverso | Revenue -100 bps, tax +200 bps | 5% |
| S6 | CAPEX Overrun | Capex adicional 25 MUSD | 8% |

**Escenarios Combinados (C1-C3):**
| Código | Nombre | Composición |
|--------|--------|-------------|
| C1 | Perfect Storm | S1 + S2 + S3 |
| C2 | Launch + Rates | S2 + S3 |
| C3 | Operational Cascade | S1 + S4 + S6 |

**Escenarios de Tokenización (T1-T3):**
| Código | Nombre | Riesgo Específico |
|--------|--------|------------------|
| T1 | DeFi Liquidity Crisis | Profundidad AMM -80% |
| T2 | Smart Contract Exploit | p=0.5% (rekt.news) |
| T3 | Security Token Ban | Regulatorio cripto |

### 5.2 Ranking por Impacto en DSCR

In [ ]:
# Visualización: Impacto de Escenarios de Estrés
PANELS['stress_impact']

In [ ]:
# Tabla detallada de escenarios
stress = RESULTS['stress_results']['ranking_by_dscr_impact']
print("=" * 70)
print("RANKING DE ESCENARIOS POR IMPACTO EN DSCR")
print("=" * 70)
print(f"{'Rank':<5} {'Código':<6} {'Nombre':<30} {'DSCR Min':<10} {'Δ vs Base':<10}")
print("-" * 70)
for i, scenario in enumerate(stress[:10], 1):
    delta_str = f"{scenario['delta']:+.2f}" if scenario['delta'] else "0.00"
    print(f"{i:<5} {scenario['code']:<6} {scenario['name'][:28]:<30} {scenario['dscr_min_stressed']:<10.3f} {delta_str:<10}")

### 5.3 Análisis de Near-Misses

Los escenarios "near-miss" se encuentran dentro del 10% del threshold de breach:

In [ ]:
near_misses = RESULTS['stress_results']['near_misses']
print("\nESCENARIOS NEAR-MISS (margen < 5% sobre covenant):")
print("-" * 50)
for nm in near_misses:
    print(f"  {nm['code']}: DSCR {nm['metric']:.3f} vs threshold {nm['threshold']:.2f} (margen {nm['margin']*100:.1f}%)")

**Hallazgos del Análisis de Estrés:**

1. **C1 (Perfect Storm)** es el escenario más severo: combina demanda débil + tasas altas + fallo de lanzamiento → DSCR mínimo 1.005x (apenas sobre default técnico)

2. **S3 (Fallo de Lanzamiento)** como factor individual tiene el mayor impacto: retrasa la rampa de ingresos, afectando el año 5 crítico

3. **Escenarios de tokenización (T1-T3)** tienen impacto neutro en DSCR: la tokenización afecta el costo de capital, no los flujos operativos

4. **Escenarios upside (U1-U3)** muestran potencial: salida de competidor (U1) eleva DSCR a 1.54x

---

## 6. Análisis AMM: Comparación V2 vs V3

### 6.1 Contexto: ¿Por qué AMM para Deuda Tokenizada?

La liquidez secundaria es crítica para reducir la prima de iliquidez. En project finance tradicional, los instrumentos de deuda son altamente ilíquidos (Bao, Pan & Wang, 2011), resultando en spreads de 50-100 bps adicionales.

Los Automated Market Makers (AMM) permiten:
- Trading 24/7 sin libro de órdenes
- Liquidez inmediata para cualquier tamaño
- Transparencia de precios on-chain

### 6.2 Modelos Evaluados

**Uniswap V2 (Constant Product):**
$$x \cdot y = k$$

La liquidez se distribuye uniformemente sobre todo el rango de precios [0, ∞), resultando en:
- Alta ineficiencia de capital
- Slippage significativo para trades grandes

**Uniswap V3 (Concentrated Liquidity):**

Los LP concentran liquidez en rangos específicos:
$$L = \frac{\Delta y}{\sqrt{P_{upper}} - \sqrt{P_{lower}}}$$

Para tokens de deuda que operan cerca de par (±5%), esto es ideal:
- Rango recomendado: ticks -500 a +500 (≈ 95.12% a 105.13% de par)
- 5x eficiencia de capital vs V2

### 6.3 Resultados de la Comparación

In [ ]:
# Visualización: Comparación V2 vs V3
PANELS['amm_comparison']

In [ ]:
# Tabla detallada de slippage
v2v3 = RESULTS['v2_v3_comparison']
print("=" * 60)
print("COMPARACIÓN DETALLADA V2 vs V3")
print("=" * 60)
print(f"\nConfiguración:")
print(f"  Notional: USD {v2v3['config']['debt_notional']/1e6:.0f}M")
print(f"  Capital LP por lado: USD {v2v3['config']['lp_capital_per_side']/1e6:.1f}M")
print(f"  Rango V3: {v2v3['config']['v3_price_range']}")

print(f"\n{'Trade Size':<15} {'V2 Slippage':<15} {'V3 Slippage':<15} {'Reducción':<15}")
print("-" * 60)
for v2_row, v3_row in zip(v2v3['v2_results'], v2v3['v3_results']):
    reduction = (abs(v2_row['slippage_pct']) - abs(v3_row['slippage_pct'])) / abs(v2_row['slippage_pct']) * 100
    print(f"{v2_row['trade_pct']:.0f}%{'':<12} {v2_row['slippage_pct']:.2f}%{'':<10} {v3_row['slippage_pct']:.2f}%{'':<10} {reduction:.1f}%")

comp = v2v3['comparison']
print(f"\n" + "=" * 60)
print("RESUMEN")
print(f"  Slippage promedio V2: {comp['avg_slippage_v2_pct']:.2f}%")
print(f"  Slippage promedio V3: {comp['avg_slippage_v3_pct']:.2f}%")
print(f"  Reducción de slippage: {comp['slippage_reduction_pct']:.1f}%")
print(f"  Eficiencia de capital: {comp['capital_efficiency_multiple']:.1f}x")

### 6.4 Justificación de la Selección de V3

**¿Por qué V3 sobre V2?**

1. **Eficiencia de Capital (5x):** Con USD 2.5M por lado, V3 logra la misma profundidad efectiva que V2 con USD 12.5M

2. **Reducción de Slippage (83%):** Para un trade del 10% del notional:
   - V2: -17.3% slippage
   - V3: -3.9% slippage

3. **Fit con Tokens de Deuda:** Los tokens de deuda project finance:
   - Operan cerca de par (rango ±5% captura >99% de movimientos)
   - Son activos estables (no especulativos)
   - Ideales para liquidez concentrada

4. **Impermanent Loss Reducido:** En rangos estrechos, IL es mínimo (0.15% vs 2% en V2)

### 6.5 Validación con Datos de Mercado (DefiLlama API)

**Metodología de validación en dos pasos:**

1. **Paso 1 - Validar precisión del modelo:** Comparamos nuestro modelo V2 contra datos reales de Tinlake/Centrifuge (que usa mecánicas tipo V2). Si el modelo V2 es preciso, podemos confiar en las extensiones.

2. **Paso 2 - Demostrar mejora de V3:** Una vez validado el modelo V2, mostramos que V3 supera a V2 (y por tanto superaría a Tinlake en producción).

In [ ]:
# Validación con datos de Tinlake/Centrifuge
amm_liq = RESULTS['amm_liquidity']
amm_rec = RESULTS['amm_recommendation']

print("=" * 70)
print("VALIDACIÓN CON DATOS DE MERCADO (DefiLlama API)")
print("=" * 70)
print(f"\nPlataforma de referencia: {amm_liq['platform']['name']}")
print(f"TVL: USD {amm_liq['platform']['tvl_usd']/1e9:.2f}B")
print(f"Nota: Tinlake/Centrifuge usa mecánicas tipo V2 (constant product)")

print("\n" + "-" * 70)
print("PASO 1: Validación del Modelo V2 contra Tinlake (datos reales)")
print("-" * 70)
comp_tinlake = amm_liq['comparison_with_tinlake']
print(f"  Tinlake benchmark (real):     {comp_tinlake['tinlake_reduction_bps']:.2f} bps de reducción")
print(f"  Nuestro modelo V2:            {comp_tinlake['amm_reduction_bps']:.2f} bps de reducción")
print(f"  Delta:                        {comp_tinlake['delta_bps']:.2f} bps")
print(f"  Validación:                   {comp_tinlake['validation'].upper()}")
print(f"  → Nuestro modelo V2 es preciso (±2 bps de datos reales)")

print("\n" + "-" * 70)
print("PASO 2: V3 supera a V2 (y por tanto a Tinlake)")
print("-" * 70)
v3_premium = amm_rec['v3_derived_premium']
v2v3_comp = amm_rec['v2_vs_v3_premium_comparison']
print(f"  V2 reducción de prima:        {v2v3_comp['v2_reduction_bps']:.2f} bps")
print(f"  V3 reducción de prima:        {v2v3_comp['v3_reduction_bps']:.2f} bps")
print(f"  Mejora V3 sobre V2:           +{v2v3_comp['improvement_bps']:.2f} bps ({v2v3_comp['improvement_pct']:.1f}%)")

print("\n" + "=" * 70)
print("CONCLUSIÓN")
print("=" * 70)
print(f"  Prima tradicional:            75.00 bps")
print(f"  Con V2 (Tinlake actual):      {75 - v2v3_comp['v2_reduction_bps']:.2f} bps")
print(f"  Con V3 (propuesta):           {v3_premium['derived_liquidity_bps']:.2f} bps")
print(f"\n  → V3 reduce la prima de liquidez de 75 bps a {v3_premium['derived_liquidity_bps']:.2f} bps")
print(f"  → Mejora de {v3_premium['reduction_bps']:.2f} bps vs tradicional")

In [ ]:
# Prima derivada de V3
amm_rec = RESULTS['amm_recommendation']
v3_premium = amm_rec['v3_derived_premium']
print("\n" + "=" * 60)
print("PRIMA DE LIQUIDEZ DERIVADA (V3)")
print("=" * 60)
print(f"  Slippage (10% trade): {v3_premium['slippage_10pct_trade_pct']:.2f}%")
print(f"  Depth score: {v3_premium['depth_score']:.4f}")
print(f"  Prima de liquidez derivada: {v3_premium['derived_liquidity_bps']:.2f} bps")
print(f"  Baseline tradicional: {v3_premium['traditional_baseline_bps']:.0f} bps")
print(f"  REDUCCIÓN TOTAL: {v3_premium['reduction_bps']:.2f} bps")

v2v3_comp = amm_rec['v2_vs_v3_premium_comparison']
print(f"\nV3 vs V2:")
print(f"  V2 reducción: {v2v3_comp['v2_reduction_bps']:.2f} bps")
print(f"  V3 reducción: {v2v3_comp['v3_reduction_bps']:.2f} bps")
print(f"  Mejora adicional: +{v2v3_comp['improvement_bps']:.2f} bps ({v2v3_comp['improvement_pct']:.1f}%)")

### 6.6 Conclusión del Análisis AMM

| Aspecto | V2 | V3 | Ganador |
|---------|----|----|--------|
| Slippage | -20.8% | -3.6% | **V3** |
| Eficiencia Capital | 1.0x | 5.0x | **V3** |
| Reducción Prima | 56 bps | 70 bps | **V3** |
| Complejidad | Baja | Media | V2 |
| Riesgo Out-of-Range | N/A | Bajo | N/A |

**Recomendación:** V3 como modelo primario para mercados secundarios de deuda tokenizada, implementado sobre Centrifuge + Uniswap V3 con fee tier de 30 bps.

---

## 7. Análisis de Cobertura (Hedging)

### 7.1 Exposición a Tasa de Interés

El proyecto tiene exposición a tasa flotante: base 4.5% + spread variable. Un shock de +200 bps reduce el DSCR en ~0.25x, aumentando significativamente la probabilidad de breach.

### 7.2 Instrumentos Evaluados

**Interest Rate Cap:**
- Notional: USD 50M
- Strike: 4.0% (100 bps OTM)
- Tenor: 5 años (caplets anuales)
- Valoración: Black-76 con σ = 20% flat

**Zero-Cost Collar:**
- Cap strike: 4.0%
- Floor strike: 3.0%
- Premio neto: ~0 (floor financia cap)

### 7.3 Resultados de la Comparación

In [ ]:
# Visualización: Comparación de Coberturas
PANELS['hedging_comparison']

In [ ]:
# Tabla detallada de hedging
hedging = RESULTS['hedging_comparison']
print("=" * 70)
print("COMPARACIÓN DE ESTRATEGIAS DE COBERTURA")
print("=" * 70)
print(f"\nConfiguración:")
print(f"  Notional: USD {hedging['hedge_config']['notional']/1e6:.0f}M")
print(f"  Cap Strike: {hedging['hedge_config']['cap_strike']*100:.1f}%")
print(f"  Floor Strike: {hedging['hedge_config']['floor_strike']*100:.1f}%")
print(f"  Tasa Base: {hedging['hedge_config']['base_rate']*100:.1f}%")

print(f"\n{'Estrategia':<15} {'Prob. Breach':<15} {'Breaches':<12} {'Costo (USD)':<15} {'Costo/Breach':<15}")
print("-" * 70)
scenarios = hedging['scenarios']
base_breaches = scenarios['none']['breach_count']
for name, data in scenarios.items():
    avoided = base_breaches - data['breach_count']
    cost_per = data['cost'] / max(avoided, 1) if avoided > 0 else 0
    print(f"{name.capitalize():<15} {data['breach_probability']*100:.2f}%{'':<10} {data['breach_count']:<12} ${data['cost']:,.0f}{'':<8} ${cost_per:,.0f}")

print(f"\nRecomendación: {hedging['recommendation'].upper()}")
print(f"Justificación: {hedging['rationale']}")

### 7.4 Análisis de la Recomendación

**¿Por qué Cap sobre Collar?**

| Métrica | Sin Hedge | Cap | Collar |
|---------|-----------|-----|--------|
| Prob. Breach | 2.71% | 2.09% | 2.38% |
| Reducción | — | -23% | -12% |
| Costo | $0 | $595K | $326K |

1. **Mayor Efectividad:** El cap reduce breach probability 23% vs 12% del collar
2. **Protección de Cola:** El cap protege contra tasas >5.5% donde el collar tiene exposición limitada
3. **Sin Restricción Downside:** El collar limita ganancias si tasas bajan <3%

**Recomendación:** Interest rate cap como parte de la estructura óptima tokenizada.

### 7.5 Impacto en WACD

In [ ]:
# Costo de hedging amortizado
cap_data = RESULTS['hedging']['interest_rate_cap']
print("\nIMPACTO DEL CAP EN WACD:")
print(f"  Prima del cap: USD {cap_data['premium']:,.0f}")
print(f"  Break-even spread: {cap_data['break_even_spread_bps']:.2f} bps")
print(f"  Carry cost (% notional): {cap_data['carry_cost_pct']:.2f}%")
print(f"  Amortizado sobre 5 años: {cap_data['premium']/5/RESULTS['hedging']['interest_rate_cap']['notional']*10000:.2f} bps/año")

---

## 8. Estructura Tradicional vs Tokenizada

### 8.1 El Problema de Bancabilidad

El hallazgo crítico del WP-08 fue que **el proyecto como estructurado originalmente (70% LTV) NO era bancable**:

- Precios estocásticos: 0.86-0.92 por par (yields implícitos 11-20%)
- Probabilidad de breach acumulada: 60%
- Rating implícito: Below Investment Grade

**Solución implementada:**
1. Deleveraging a 50% LTV (USD 50M deuda vs USD 72M original)
2. Amortización contingente basada en DSCR

### 8.2 Mecanismo de Amortización Contingente

La innovación clave de la estructura tokenizada es la **amortización contingente**:

```
SI DSCR < 1.25x (floor):
    → Diferir hasta 30% del principal programado
    → Acumular como balloon al final del tenor
    → Balloon cap: 50% del principal original
    → Si balloon > 50%: extensión 5 años @ spread +200 bps

SI DSCR > 2.0x (accelerate):
    → Amortización acelerada del balloon diferido
```

Este mecanismo es posible gracias a smart contracts que ejecutan la lógica automáticamente, sin necesidad de negociación con acreedores.

### 8.3 Resultados de la Comparación

In [ ]:
# Comparación detallada de estructuras
dual = RESULTS['dual_structure_comparison']
print("=" * 70)
print("COMPARACIÓN ESTRUCTURA TRADICIONAL vs TOKENIZADA")
print("=" * 70)
print(f"Simulaciones: {dual['simulations']:,}")
print(f"Covenant DSCR: {dual['covenant']:.2f}x")

print(f"\n{'Métrica':<30} {'Tradicional':<20} {'Tokenizada':<20}")
print("-" * 70)
trad = dual['traditional']
tok = dual['tokenized']
print(f"{'Probabilidad de Breach':<30} {trad['breach_probability']*100:.2f}%{'':<15} {tok['breach_probability']*100:.2f}%")
print(f"{'Conteo de Breaches':<30} {trad['breach_count']:,}{'':<17} {tok['breach_count']:,}")
print(f"{'DSCR Mínimo (P5)':<30} {trad['min_dscr_p5']:.3f}{'':<17} {tok['min_dscr_p5']:.3f}")
print(f"{'DSCR Mínimo (P25)':<30} {trad['min_dscr_p25']:.3f}{'':<17} {tok['min_dscr_p25']:.3f}")
print(f"{'DSCR Mínimo (P50)':<30} {trad['min_dscr_p50']:.3f}{'':<17} {tok['min_dscr_p50']:.3f}")
print(f"{'Año Promedio de Breach':<30} {trad['avg_breach_year']:.1f}{'':<18} {tok['avg_breach_year']:.1f}")

if 'avg_additional_balloon' in tok:
    print(f"{'Balloon Adicional Promedio':<30} N/A{'':<18} USD {tok['avg_additional_balloon']/1e6:.2f}M")
    print(f"{'Balloon Máximo':<30} N/A{'':<18} USD {tok['max_additional_balloon']/1e6:.2f}M")

In [ ]:
# Impacto de la tokenización
comp = dual['comparison']
print("\n" + "=" * 70)
print("IMPACTO DE LA TOKENIZACIÓN")
print("=" * 70)
print(f"  Reducción de prob. breach: {comp['breach_probability_reduction']*100:.1f} puntos porcentuales")
print(f"  Reducción relativa: {comp['breach_probability_reduction_pct']:.1f}%")
print(f"  Breaches evitados: {comp['breaches_avoided_count']:,} ({comp['breaches_avoided_pct']:.1f}%)")
print(f"  Mejora DSCR P25: +{comp['dscr_p25_improvement']:.4f}")

print(f"\n  CONCLUSIÓN CLAVE:")
print(f"  → Tradicional bancable: NO")
print(f"  → Tokenizado bancable: SÍ")
print(f"  → La amortización contingente basada en DSCR reduce la probabilidad")
print(f"    de breach de 24.6% a 2.7% (reducción del 89%), transformando")
print(f"    el proyecto de no-bancable a bancable.")

### 8.4 Visualización de Estructura de Capital

In [ ]:
# Visualización: Estructura de Capital
PANELS['capital_structure']

### 8.5 Implicaciones para la Tesis

La comparación demuestra que:

1. **La tokenización habilita mecanismos de flexibilidad** que serían costosos o imposibles en estructuras tradicionales (negociación con sindicato de bancos)

2. **La amortización contingente es el driver principal** de la mejora en bancabilidad, reduciendo breach de 24.6% a 2.7%

3. **El DSCR floor de 1.25x** actúa como "airbag" preventivo, evitando cascadas de default

4. **El balloon promedio de USD 1.3M** es manejable (2.7% del principal), indicando que el mecanismo se activa moderadamente

---

## 9. Síntesis WACD

### 9.1 Componentes del WACD All-In

El WACD (Weighted Average Cost of Debt) sintetiza todos los componentes analizados:

In [ ]:
# Visualización: Síntesis WACD
PANELS['wacd_synthesis']

In [ ]:
# Descomposición de componentes
wacd = RESULTS['wacd_synthesis']
print("=" * 70)
print("DESCOMPOSICIÓN DE COMPONENTES WACD")
print("=" * 70)
comp = wacd['components']
print(f"\nBeneficios de Tokenización:")
print(f"  Reducción liquidez (V3 AMM): -{comp['liquidity_reduction_bps']:.2f} bps")
print(f"  Reducción operacional: -{comp['operational_reduction_bps']:.2f} bps")
print(f"  Reducción transparencia: -{comp['transparency_reduction_bps']:.2f} bps")
print(f"  TOTAL beneficio tokenización: -{comp['total_tokenization_reduction_bps']:.2f} bps")
print(f"\nCostos Adicionales:")
print(f"  Prima riesgo regulatorio: +{comp['regulatory_risk_bps']:.2f} bps")
print(f"  Costo de hedging (cap): +{comp['hedging_cost_bps']:.2f} bps")

In [ ]:
# Escenarios WACD
print("\n" + "=" * 70)
print("ESCENARIOS WACD ALL-IN")
print("=" * 70)
print(f"{'Escenario':<35} {'Cupón':<10} {'Beneficio':<12} {'Hedge':<10} {'All-In':<10}")
print("-" * 70)
for name, scenario in wacd['scenarios'].items():
    print(f"{name:<35} {scenario['coupon_wacd_bps']:<10.0f} {scenario['tokenization_benefit_bps']:<12.2f} {scenario['hedging_cost_bps']:<10.2f} {scenario['all_in_wacd_bps']:<10.2f}")

savings = wacd['comparison_vs_traditional']
print(f"\nAhorros vs Tradicional:")
print(f"  Misma estructura: {savings['tokenized_same_structure_savings_bps']:.2f} bps")
print(f"  Estructura óptima: {savings['tokenized_optimal_savings_bps']:.2f} bps")
print(f"  Óptima + hedge: {savings['tokenized_optimal_hedged_savings_bps']:.2f} bps")

### 9.2 Hallazgos Clave del WACD

In [ ]:
# Hallazgos de la tesis
thesis = wacd['thesis_finding']
print("\n" + "=" * 70)
print("HALLAZGOS DE LA TESIS - WACD")
print("=" * 70)
for key, value in thesis.items():
    print(f"\n{key.replace('_', ' ').title()}:")
    print(f"  {value}")

---

## 10. Análisis de Equity

### 10.1 Estructura de Capital y Retornos

In [ ]:
# Análisis de equity
equity = RESULTS['equity_analysis']
print("=" * 60)
print("ANÁLISIS DE EQUITY")
print("=" * 60)

cap = equity['capital_structure']
print(f"\nEstructura de Capital:")
print(f"  Costo total proyecto: USD {cap['total_project_cost_musd']:.0f}M")
print(f"  Deuda: USD {cap['debt_musd']:.0f}M ({cap['debt_pct']:.0f}%)")
print(f"  Equity: USD {cap['equity_musd']:.0f}M ({cap['equity_pct']:.0f}%)")

ret = equity['returns']
print(f"\nRetornos al Equity:")
print(f"  IRR: {ret['equity_irr_pct']:.2f}%")
print(f"  Múltiplo: {ret['multiple']:.2f}x")
print(f"  Payback: Año {ret['payback_year']}")
print(f"  Dividendos totales: USD {ret['total_dividends_musd']:.2f}M")

cf = equity['cashflows']
print(f"\nFlujos de Equity:")
print(f"  Inversión inicial: USD {cf['initial_investment_musd']:.0f}M")
print(f"  Dividendos por año:")
for i, div in enumerate(cf['dividends_by_year'], 1):
    if div > 0:
        print(f"    Año {i}: USD {div:.2f}M")

### 10.2 Costo de Oportunidad LP

Si los sponsors proveyeran capital LP en lugar de usar Centrifuge:

In [ ]:
lp = equity['lp_opportunity_cost']
print(f"\nCosto de Oportunidad LP:")
print(f"  {lp['note']}")
print(f"  Capital LP requerido: USD {lp['lp_capital_musd']:.1f}M")
print(f"  Costo oportunidad anual: USD {lp['annual_opportunity_cost_musd']:.2f}M")
print(f"  vs yield stablecoin: {lp['vs_stablecoin_yield_pct']:.1f}%")
print(f"\n  → Usando Centrifuge se evita USD {lp['annual_opportunity_cost_musd']*15:.2f}M en costo oportunidad (15 años)")

---

## 11. Conclusiones y Recomendaciones

### 11.1 Respuesta a la Pregunta de Investigación

**Pregunta:** ¿Puede la tokenización de instrumentos de deuda en project finance reducir significativamente el costo de capital y mejorar la bancabilidad?

**Respuesta:** **Sí, de manera cuantificable y significativa.**

### 11.2 Hallazgos Principales

| Dimensión | Impacto Cuantificado | Mecanismo |
|-----------|---------------------|----------|
| **Bancabilidad** | 24.6% → 2.1% breach prob. (-91%) | Amortización contingente + Cap |
| **Costo de Deuda** | 450 → 388 bps (-62 bps netos) | Reducción fricciones - costo hedge |
| **Prima Liquidez** | 75 → 5 bps (-93%) | AMM V3 concentrado |
| **Eficiencia Capital** | 1x → 5x (+400%) | Liquidez concentrada |
| **Riesgo de Tasa** | 2.71% → 2.09% breach | Interest rate cap |

### 11.3 Estructura Óptima Recomendada

La estructura tokenizada óptima combina tres mecanismos:

1. **Amortización Contingente:** Diferimiento automático de principal cuando DSCR < 1.25x
2. **AMM V3:** Liquidez concentrada en rango ±5% para mercado secundario
3. **Interest Rate Cap:** Strike 4.0%, tenor 5 años, costo ~24 bps/año

### 11.4 Contribuciones Metodológicas

1. **Framework de Decomposición de Spreads:** Metodología para cuantificar cada componente del spread tokenizado vs tradicional

2. **Modelo AMM V3 para RWA:** Primera aplicación documentada de liquidez concentrada Uniswap V3 a deuda de project finance

3. **Amortización Contingente:** Mecanismo de flexibilidad habilitado por smart contracts que transforma proyectos no-bancables en bancables

4. **Validación con Datos de Mercado:** Calibración contra DefiLlama/Tinlake con TVL real de USD 1.45B

### 11.5 Limitaciones

1. **Caso único:** Resultados específicos a constelación LEO IoT; generalización requiere casos adicionales

2. **Riesgo regulatorio:** Prima de 7.5 bps asume probabilidad 5% de ban; escenarios adversos podrían ser peores

3. **Modelo single-period Merton:** Simplificación vs modelos path-dependent de crédito

4. **Volatilidad implícita flat:** Black-76 con σ=20% constante; smile/skew no modelado

### 11.6 Trabajo Futuro

1. **Extensión a múltiples proyectos:** Aplicar framework a energía renovable, agua, transporte

2. **Modelo KMV path-dependent:** Refinar PD con estructura temporal

3. **Optimización de rangos V3:** Rebalanceo automático con Chainlink Keepers

4. **Análisis de correlación cross-project:** Portafolios de deuda tokenizada diversificados

---

## 12. Referencias Bibliográficas

### Project Finance

- Gatti, S. (2018). *Project Finance in Theory and Practice: Designing, Structuring, and Financing Private and Public Projects*. Academic Press.

- Yescombe, E.R. (2013). *Principles of Project Finance*. Academic Press.

- Esty, B.C. & Sesia, A. (2011). An Overview of Project Finance and Infrastructure Finance. *Harvard Business School Background Note* 214-083.

### Riesgo de Crédito

- Merton, R.C. (1974). On the Pricing of Corporate Debt: The Risk Structure of Interest Rates. *Journal of Finance*, 29(2), 449-470.

- Artzner, P., Delbaen, F., Eber, J.M., & Heath, D. (1999). Coherent Measures of Risk. *Mathematical Finance*, 9(3), 203-228.

- Basel Committee on Banking Supervision. (2009). *Enhancements to the Basel II Framework*. Bank for International Settlements.

- Moody's Investors Service. (2022). *Default and Recovery Rates for Project Finance Bank Loans, 1983-2020*.

### DeFi y Tokenización

- Schär, F. (2021). Decentralized Finance: On Blockchain- and Smart Contract-Based Financial Markets. *Federal Reserve Bank of St. Louis Review*, 103(2), 153-174.

- Zetzsche, D.A., Arner, D.W., & Buckley, R.P. (2020). Decentralized Finance. *Journal of Financial Regulation*, 6(2), 172-203.

- Makarov, I. & Schoar, A. (2022). Cryptocurrencies and Decentralized Finance. *NBER Working Paper* No. 30006.

### Microestructura de Mercado

- Bao, J., Pan, J., & Wang, J. (2011). The Illiquidity of Corporate Bonds. *Journal of Finance*, 66(3), 911-946.

- Amihud, Y. (2002). Illiquidity and Stock Returns: Cross-Section and Time-Series Effects. *Journal of Financial Markets*, 5(1), 31-56.

### Derivados de Tasa

- Black, F. (1976). The Pricing of Commodity Contracts. *Journal of Financial Economics*, 3(1-2), 167-179.

### Industria Satelital

- Jakhu, R.S. & Pelton, J.N. (Eds.). (2017). *Global Space Governance: An International Study*. Springer.

- IEEE Aerospace. (2021). LEO Constellation Lifecycle Studies.

### Fuentes de Datos

- DefiLlama API: https://api.llama.fi/protocol/centrifuge
- Centrifuge Documentation: https://docs.centrifuge.io
- Uniswap V3 Whitepaper: https://uniswap.org/whitepaper-v3.pdf
- SEC EDGAR: Iridium Communications 10-K (2019-2023)
- SpaceX Reliability Data: https://www.spacex.com/reliability

---

## 13. Reproducibilidad

### 13.1 Entorno de Ejecución

In [ ]:
import sys
import platform
from datetime import datetime

print("=" * 60)
print("INFORMACIÓN DE REPRODUCIBILIDAD")
print("=" * 60)
print(f"\nEntorno:")
print(f"  Python: {sys.version}")
print(f"  Plataforma: {platform.system()} {platform.release()}")
print(f"  Fecha de ejecución: {datetime.now().isoformat()}")

print(f"\nDatos:")
print(f"  Archivo: {RESULTS_PATH}")
print(f"  Timestamp: {RESULTS.get('timestamp_utc', 'N/A')}")

mc_config = RESULTS.get('monte_carlo', {}).get('config', {})
print(f"\nConfiguración Monte Carlo:")
print(f"  Simulaciones: {mc_config.get('simulations', 'N/A'):,}")
print(f"  Semilla: {mc_config.get('seed', 'N/A')}")
print(f"  Variantes antitéticas: {mc_config.get('antithetic', 'N/A')}")

In [ ]:
# Versiones de paquetes clave
import importlib.metadata

packages = ['numpy', 'pandas', 'scipy', 'plotly', 'jupyter']
print("\nVersiones de Paquetes Clave:")
for pkg in packages:
    try:
        version = importlib.metadata.version(pkg)
        print(f"  {pkg}: {version}")
    except importlib.metadata.PackageNotFoundError:
        print(f"  {pkg}: no instalado")

### 13.2 Comandos de Regeneración

Para regenerar los resultados desde cero:

```bash
# 1. Regenerar JSON de resultados
python scripts/demo_risk_metrics.py

# 2. Exportar dashboard HTML
python -c "from pftoken.viz.plotly_panels import *; import json; \
    results = json.load(open('outputs/leo_iot_results.json')); \
    panels = build_interactive_dashboard(results); \
    export_dashboard_html(panels, 'outputs/dashboards/wp09_dashboard.html')"

# 3. Exportar notebook a HTML
jupyter nbconvert --to html notebooks/TP_Quant_Final.ipynb --output-dir=outputs/dashboards/
```

### 13.3 Checksum de Datos

In [ ]:
import hashlib

with open(RESULTS_PATH, 'rb') as f:
    checksum = hashlib.sha256(f.read()).hexdigest()[:16]
print(f"\nChecksum (SHA256, primeros 16 chars): {checksum}")
print("\n" + "=" * 60)
print("FIN DEL NOTEBOOK")
print("=" * 60)

---

**© 2025 - Trabajo de Tesis - Maestría en Finanzas Cuantitativas**

*Generado con el framework de valoración cuantitativa de project finance tokenizado.*